In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import spacy
import re
import nltk
import string
import sklearn
import tensorflow as tf
from nltk.corpus import stopwords
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:546: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 KB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 KB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 7.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17413 sha256=5e7a716c40e5e78dda9bd184cdca54a3879253cc71b6e06800e98c46e4d07b79
  Stored in directory: /root/.cache/pip/wheels/60/b3/27/d8aff3e2d5c2d0d97a117cdf0d5f13cd121e2c2b5fb49b55a0
Successfully built googletrans

In [ ]:
!pip install SpeechRecognition pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 39.9 MB/s eta 0:00:00


In [ ]:
rd=pd.read_csv("/content/EDA_PP_F.csv",encoding='latin1')
rd.head()

,Unnamed: 0,tweet_text,type
0,0,in words food crapilicious,none
1,2,classy whore or red velvet cupcakes,none
2,3,meh thanks heads up concerned angry dude twitter,none
3,4,this isis account pretending kurdish account l...,none
4,5,yes test god good bad indifferent weird whatev...,none


In [ ]:
label_enc = LabelEncoder()
rd['type'] = label_enc.fit_transform(rd['type'])

In [ ]:
X = rd['tweet_text']
y = rd['type']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
np.asarray((unique, counts)).T

array([[   0, 6300],
       [   1, 5204],
       [   2, 6179],
       [   3, 6331],
       [   4, 5843]])

In [ ]:
ros = RandomOverSampler()
X_train, y_train = ros.fit_resample(np.array(X_train).reshape(-1, 1), np.array(y_train).reshape(-1, 1));
train_os = pd.DataFrame(list(zip([x[0] for x in X_train], y_train)), columns = ['tweet_text', 'type']);

In [ ]:
X_train = train_os['tweet_text'].values
y_train = train_os['type'].values

In [ ]:
(unique, counts) = np.unique(y_train, return_counts=True)
np.asarray((unique, counts)).T

array([[   0, 6331],
       [   1, 6331],
       [   2, 6331],
       [   3, 6331],
       [   4, 6331]])

In [ ]:
cv = CountVectorizer()
X_train_cv =  cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

In [ ]:
tf_transformer = TfidfTransformer(use_idf=True).fit(X_train_cv)
X_train_tf = tf_transformer.transform(X_train_cv)
X_test_tf = tf_transformer.transform(X_test_cv)

In [ ]:
rf = RandomForestClassifier(random_state=42)
# gb = GradientBoostingClassifier(random_state=42)
# ada = AdaBoostClassifier(random_state=42)
# lgb = LGBMClassifier(random_state=42)
# xgb = XGBClassifier(eval_metric="mlogloss",random_state=42)
# dt = DecisionTreeClassifier(random_state=42)
# svc = SVC(random_state=42)
# nb = MultinomialNB()
# mlp = MLPClassifier(random_state=42)

models = {
    "Random Forest": rf,
    # "Gradient Boosting":gb,
    # "AdaBoost": ada,
    # "LightGBM": lgb,
    # "XGBoost": xgb,
    # "Decision Tree":dt,
    # "Support Vector Machine":svc,
    # "Naive Bayes": nb,
    # "Multilayer Perceptron":mlp
}

In [ ]:
def fit_model(clf,x_train,y_train,x_test, y_test):
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    accuracy = accuracy_score(y_pred, y_test)
    return accuracy

accuracys = []


for name,clf in models.items():
    curr_acc = fit_model(clf,X_train_tf,y_train,X_test_tf,y_test)
    accuracys.append(curr_acc)

In [ ]:
# models_df = pd.DataFrame({"Models":models.keys(),"Accuracy Scores":accuracys}).sort_values('Accuracy Scores',ascending=False)
# models_df

In [ ]:
sentiments = ["age","not bullying","racism","religion","gender"]
all_categories_names=np.array(sentiments)
x=rf.predict(cv.transform(["hi hello"]))
print(all_categories_names[x])

['not bullying']


In [ ]:
# x=rf.predict(cv.transform([translated.text]))
# print(all_categories_names[x])

In [ ]:
# all imports
from io import BytesIO
from base64 import b64decode
from google.colab import output
from IPython.display import Javascript

RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=3):
  print("Speak Now...")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

In [ ]:
def record_lang(sec=3):
  print("Please tell us the language of input")
  display(Javascript(RECORD))
  sec += 1
  s = output.eval_js('record(%d)' % (sec*1000))
  print("Done Recording !")
  b = b64decode(s.split(',')[1])
  return b #byte stream

In [ ]:
from io import BytesIO
from pydub import AudioSegment

def save_audio_file(byte_stream, filename):
    # read the byte stream into an audio segment
    audio_segment = AudioSegment.from_file(BytesIO(byte_stream))

    # set the frame rate of the audio segment
    audio_segment = audio_segment.set_frame_rate(16000)

    # export the audio segment to a file
    audio_segment.export(filename, format="wav")

    print("Audio file saved as:", filename)

In [ ]:
byte_stream = record_lang(3)
save_audio_file(byte_stream, "language.wav")

In [ ]:
import speech_recognition as sr
filename = "/content/language.wav"
# initialize the recognizer
r = sr.Recognizer()
# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    lang_text = r.recognize_google(audio_data)
    print(lang_text)

In [ ]:
byte_stream = record(5)
save_audio_file(byte_stream, "my_audiofile.wav")

Speak Now...


<IPython.core.display.Javascript object>

Done Recording !
Audio file saved as: my_audiofile.wav


In [ ]:
lang=''
if lang_text == 'french':
  lang="fr-FR"
elif lang_text == 'spanish':


In [ ]:
import speech_recognition as sr
filename = "/content/my_audiofile.wav"
# initialize the recognizer
r = sr.Recognizer()
# open the file
with sr.AudioFile(filename) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data,language = lang)
    print(text)

salut tu es une pute


In [ ]:
from googletrans import Translator
translator=Translator()
translated=translator.translate(text)
print(translated.text)

hi you are a whore


In [ ]:
x=rf.predict(cv.transform(["hi are u a nigger"]))
print(all_categories_names[x])

['racism']


In [ ]:
#Adding french langauge option
#text = r.recognize_google(audio_data, language = "fr-FR")